In [27]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup

import pandas as pd

import time

from datetime import date
from dateutil.rrule import rrule, DAILY, MONTHLY

import os.path
import os

In [28]:
def initial_tasks():
    profile = webdriver.FirefoxProfile()
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.download.dir", 'C:\betfairData')
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-tar, application/x-download")

    driver = webdriver.Firefox(firefox_profile=profile)

    driver.get('https://historicdata.betfair.com')
    driver.find_element_by_name('username').send_keys('username')
    driver.find_element_by_name('password').send_keys('password')
    driver.find_element_by_tag_name('body').send_keys(Keys.ENTER)

    driver.find_element_by_xpath("//a[contains(@href, '#/mydata')]").click()

    time.sleep(2)

    driver.get('https://historicdata.betfair.com/#/mydata')

    try:
        elem = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '//div/xdw-plan-drop-down/select')))
    except TimeoutException:
        print("Too much time")

    driver.execute_script("window.open('http://google.com', 'new_window')")
    driver.switch_to_window(driver.window_handles[1])

    driver.get('about:downloads')
    driver.switch_to_window(driver.window_handles[0])
    return driver

In [29]:
def empty_cart(driver):
    while True:
        try:
            time.sleep(1)
            driver.implicitly_wait(1)
            driver.find_element_by_css_selector('div.red-cross').click()
            driver.implicitly_wait(30)
        except:
            break

In [30]:
def perform_download(driver, dt, oldday, oldmonth, oldyear):
    driver.switch_to_window(driver.window_handles[0])

    driver.get('https://historicdata.betfair.com/#/mydata')
    time.sleep(2)


    Select(driver.find_element_by_xpath('//div/xdw-plan-drop-down/select')).select_by_value('Basic Plan')

    Select(driver.find_element_by_xpath('//div/xdw-sport-drop-down/select')).select_by_value('Soccer')

    Select(driver.find_element_by_xpath('//xdw-day-drop-down/select')).select_by_value(str(oldday))
    Select(driver.find_element_by_xpath('//div/xdw-month-drop-down/select')).select_by_index(str(oldmonth))
    Select(driver.find_element_by_xpath('//div/xdw-year-drop-down/select')).select_by_value(str(oldyear))

    Select(driver.find_element_by_xpath('//div[2]/xdw-day-drop-down/select')).select_by_value(str(dt.day))
    Select(driver.find_element_by_xpath('//div[2]/xdw-month-drop-down/select')).select_by_index(str(dt.month))
    Select(driver.find_element_by_xpath('//div[2]/xdw-year-drop-down/select')).select_by_value(str(dt.year))
    
    empty_cart(driver)
    print("Cart emptied")
    

    driver.find_element_by_xpath('//td[2]/button').click() #Click add to downloads
    
    
    while True: #to verify download button opened
        time.sleep(2)
        if "You are downloading" in driver.find_element_by_tag_name('body').text:
            break
        else:
            empty_cart(driver)
            time.sleep(2)
            driver.find_elements_by_tag_name('button')[3].click()
            time.sleep(1)
            
    print("Added to downloads")

    time.sleep(3)
    
    #click the download button here
    getURL = driver.find_element_by_partial_link_text('Download').get_attribute('href')
    
    try:
        driver.get(getURL)
        time.sleep(7)
    except:
        pass
    
    print("URL Opened")
    
    driver.switch_to_window(driver.window_handles[1])

    state = '0'
    count = 0
    
    while True: #to verify download started
        try:
            state = driver.find_element_by_tag_name('richlistitem').get_attribute('state')
            break
        except:
            count+=1
            
            print("Not downloading")
            
            if count > 5:
                print('Some Error. Retrying')
                driver.quit()
                driver = initial_tasks()
                perform_download(driver, dt, oldday, oldmonth, oldyear)
            else:
                driver.switch_to_window(driver.window_handles[0])
                try:
                    driver.find_element_by_css_selector('a.a-as-black-on-glod-button.pull-right').click()
                except:
                    pass

                driver.switch_to_window(driver.window_handles[1])
    print("Downloading")
    
    driver.switch_to_window(driver.window_handles[0])
    
    empty_cart(driver)
    print("Emptying cart")
    
    driver.switch_to_window(driver.window_handles[1])

    while state == '0':
        time.sleep(1)
        state = driver.find_element_by_tag_name('richlistitem').get_attribute('state')
                
    print("Download Completed")
    

    time.sleep(10) #to allow download to start
    driver.switch_to_window(driver.window_handles[0])

    time.sleep(5)
    os.rename('C:/betfairData/data.tar', 'C:/betfairData/{}-{}-{}-{}-{}-{}.tar'.format(oldyear, oldmonth, oldday, dt.year, dt.month, dt.day))
    print("Renamed")
    return driver

In [31]:
a = date(2015, 12, 1)
b = date(2018, 3, 1)

oldday = 1
oldmonth = 11
oldyear = 2015

driver = initial_tasks()

for dt in rrule(MONTHLY, dtstart=a, until=b):
    print("Scraping from {}/{}/{} to {}/{}/{}".format(oldyear, oldmonth, oldday, dt.year, dt.month, dt.day))
        
    driver = perform_download(driver, dt, oldday, oldmonth, oldyear)
        
    oldday = dt.day
    oldmonth = dt.month
    oldyear = dt.year

Scraping from 2015/11/1 to 2015/12/1
Cart emptied
Added to downloads
URL Opened
Downloading
Emptying cart
Download Completed
Renamed
Scraping from 2015/12/1 to 2016/1/1
Cart emptied
Added to downloads
URL Opened
Downloading
Emptying cart
Download Completed
Renamed
Scraping from 2016/1/1 to 2016/2/1
Cart emptied
Added to downloads
URL Opened
Downloading
Emptying cart
Download Completed
Renamed
Scraping from 2016/2/1 to 2016/3/1
Cart emptied
Added to downloads
URL Opened
Downloading
Emptying cart
Download Completed
Renamed
Scraping from 2016/3/1 to 2016/4/1
Cart emptied
Added to downloads
URL Opened
Downloading
Emptying cart
Download Completed
Renamed
Scraping from 2016/4/1 to 2016/5/1
Cart emptied
Added to downloads
URL Opened
Downloading
Emptying cart
Download Completed
Renamed
Scraping from 2016/5/1 to 2016/6/1
Cart emptied
Added to downloads
URL Opened
Downloading
Emptying cart
Download Completed
Renamed
Scraping from 2016/6/1 to 2016/7/1
Cart emptied
Added to downloads
URL Opened
Do

In [ ]:
#redownload missing files

a = date(2015, 5, 1)
b = date(2018, 5, 1)

oldday = 1
oldmonth = 4
oldyear = 2015

driver = initial_tasks()

for dt in rrule(MONTHLY, dtstart=a, until=b):
    fname =  'C:/betfairData/{}-{}-{}-{}-{}-{}.tar'.format(oldyear, oldmonth, oldday, dt.year, dt.month, dt.day)
    redo = 0
    
    if (os.path.isfile(fname)):
        if (os.stat(fname).st_size < 1):
            print("Deleting {} because 0 KB".format(fname))
            os.remove(fname)
            redo = 1
    else:
        redo = 1
        
    if (redo == 1):
        print("Scraping from {}/{}/{} to {}/{}/{}".format(oldyear, oldmonth, oldday, dt.year, dt.month, dt.day))
        driver = perform_download(driver, dt, oldday, oldmonth, oldyear)
        
    oldday = dt.day
    oldmonth = dt.month
    oldyear = dt.year

Deleting C:/betfairData/2015-11-1-2015-12-1.tar because 0 KB
Scraping from 2015/11/1 to 2015/12/1
Cart emptied
Added to downloads
URL Opened
Downloading
Emptying cart
